# [실습1] 구조를 개선한 GAN 모델

### 학습목표
1. Vanilla GAN의 구조를 개선하여 성능이 향상된 모델의 종류를 학습합니다.

2. 다양한 GAN 모델의 특징을 이해합니다.

## 목차
1. DCGAN
2. cGAN

## 1. DCGAN

GAN은 2014년 Ian Goodfellow가 발표한 이후 다양한 구조로 발전해 왔습니다. 이번 실습에서는 현재까지 발표된 GAN 모델 중 일부의 구조와 생성 결과에 대해 직접 실행하며 알아봅니다.

첫 번째로 알아볼 모델은 **DCGAN**(**D**eep **C**onvolutional **G**enerative **A**dversarial **N**etworks)입니다. **DCGAN**은 기본적인 GAN(Vanilla GAN)과 달리 CNN 기법을 사용하여 이미지의 구조적 특징을 더 효과적으로 학습합니다.

- `합성곱 신경망 구조(Convolutional Neural Network)` : DCGAN은 합성곱 레이어를 사용하여 이미지 데이터에 대한 학습을 진행합니다.
- `배치 정규화(Batch Normalization)` : DCGAN은 모든 레이어에 배치 정규화를 적용하여 학습의 안정성을 높이고, 학습 속도를 빠르게 만들어 줍니다.
- `완전 연결 층(Fully Connected Layer) 제거` : DCGAN은 이미지 생성 과정에서 전통적인 완전 연결 층 대신에 전치 합성곱 층을 사용하여 고해상도의 이미지를 생성할 수 있습니다.
- `활성화 함수(Activation fucntions)` : DCGAN에서는 LeakyReLU 활성화 함수를 주로 사용하여 기울기 소실 문제를 완화합니다. 또한, 생성자의 마지막 레이어에서는 tanh 활성화 함수를 사용해 -1과 1 사이의 출력 범위를 갖게 합니다.

최근에 발표한 대부분의 GAN모델은 DCGAN 기반의 GAN 모델인 경우가 많습니다.

<center><img src="img/DCGAN_Structure.png" style="width: 800px"></center>

In [5]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import RandomSampler, DataLoader, Dataset
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
torch.use_deterministic_algorithms(True)

# 학습이 진행되는 과정을 확인하기 위해 고정된 SEED를 설정합니다
seed = random.randint(1, 10000) 

random.seed(seed)
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)

사용하는 데이터는 [Celeba 데이터](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)로 20만 개 이상의 유명인 얼굴 이미지 데이터 세트입니다.

In [11]:
# 데이터셋의 경로
dataroot = r"C:\lsc\GAN\celba"

# dataloader에서 사용할 스레드 수
workers = 2

# 배치 크기
batch_size = 128

# 이미지 크기
image_size = 64

# 이미지의 채널 수 (컬러 이미지이기 때문에 3으로 설정)
nc = 3

# 잠재공간 벡터의 크기
nz = 100

# 생성자를 통과하는 특징 데이터들의 채널 크기
ngf = 64

# 판별자를 통과하는 특징 데이터들의 채널 크기
ndf = 64

### 1.1 데이터 세트 불러오기

`dataset`와 `dataloader`를 설정하여 데이터 세트를 불러옵니다.

In [12]:
# dataset을 설정합니다
# image_size로 이미지의 크기를 조정하고 이미지를 [-1, 1] 범위로 정규화합니다
dataset = dset.ImageFolder(
    root=dataroot,
    transform=transforms.Compose(
        [
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    ),
)

# 학습에 사용될 데이터의 갯수를 5000개로 설정합니다.
sampler = RandomSampler(dataset, replacement=False, num_samples=5000)

# dataloader를 설정합니다
dataloader = DataLoader(
    dataset, batch_size=batch_size, sampler = sampler, shuffle=False, num_workers=workers
)

device = torch.device("cuda:0")

사용하는 이미지를 dataloader를 통해 불러와 확인해 볼까요?

In [ ]:
real_batch = next(iter(dataloader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(
    np.transpose(
        vutils.make_grid(
            real_batch[0].to(device)[:64], padding=2, normalize=True
        ).cpu(),
        (1, 2, 0),
    )
)
plt.show()

### 1.2 가중치 초기화

DCGAN의 저자들이 추천하는 정규분포를 사용하여 생성자와 판별자를 초기화합니다. 
- **평균** : 0(`mean = 0`)
- **분산** : 0.01(`stdev = 0.02`)

In [2]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### 1.3 생성자
DCGAN의 생성자는 잠재 공간 벡터 z를 입력받아 이미지를 생성하는 네트워크입니다. 네트워크의 각 층은 **전치 합성곱 층**으로 구성되어 있으며 스트라이드는 2, 활성화 함수로는 **ReLU**를 사용합니다. 마지막 출력층에서는 **tanh** 함수를 사용하여 출력 값을 -1에서 1 사이의 값으로 조정합니다.


In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # 입력층
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
           
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
        
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            # 출력층
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh(),
        )

    def forward(self, input):
        return self.main(input)

`Generator`클래스를 사용해서 생성자를 만들어볼까요?

In [ ]:
netG = Generator(1).to(device)
netG.apply(weights_init)

### 1.4 판별자

DCGAN의 판별자는 입력 이미지의 진짜, 가짜 여부를 판별하는 네트워크입니다. 판별자의 마지막 출력층에서 **Sigmoid**함수를 사용하여 **0에서 1 사이**의 값으로 조정합니다. 이때 0에 가까울수록 가짜 이미지로 판별한 것이고, 1에 가까울수록 진짜 이미지로 판별한 것을 의미합니다. 판별자는 활성화 함수로 LeakyReLU를 사용해서 Vanilla GAN의 기울기 소실 문제를 해결했습니다.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # 입력층
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 출력층
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, input):
        return self.main(input)

생성자와 마찬가지로 `Discriminator` 클래스를 사용해서 판별자를 만듭니다.

In [ ]:
netD = Discriminator(1).to(device)
netD.apply(weights_init)

### 1.5 손실함수와 옵티마이저

이제 손실함수와 옵티마이저를 설정합니다. DCGAN은 손실함수로 `Binary Cross Entropy loss`를 사용합니다.

In [ ]:
# 옵티마이저의 학습률
lr = 0.0002

# Adam 옵티마이저의 beta1 하이퍼파라미터
beta1 = 0.5

In [ ]:
# `BCELoss` 함수의 인스턴스를 초기화합니다
criterion = nn.BCELoss()

# 생성자의 학습상태를 확인할 잠재 공간 벡터를 생성합니다
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# 실제 이미지일 경우 1, 가짜 이미지일 경우 0으로 설정합니다
real_label = 1.0
fake_label = 0.0

# G와 D에서 사용할 Adam옵티마이저를 생성합니다
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

### 1.6 학습

이제 위에서 생성한 생성자, 판별자와 손실함수, 옵티마이저를 사용해서 학습을 진행하고 결과를 확인합니다.

In [ ]:
# 학습할 에폭 수
num_epochs = 10

In [ ]:
# 학습상태를 체크하기 위해 손실값들을 저장합니다
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
for epoch in range(num_epochs):
    print("Starting epoch {}...".format(epoch))
    for i, data in enumerate(dataloader, 0):
        ############
        # 판별자 학습 #
        ############
        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        # 실제 이미지를 판별자에 입력합니다
        output = netD(real_cpu).view(-1)
        # 손실값을 구합니다
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # 생성자에 사용할 잠재공간 벡터(노이즈)를 생성합니다
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # 생성자에 노이즈를 입력하여 이미지를 생성합니다
        fake = netG(noise)
        label.fill_(fake_label)
        # 생성된 이미지를 판별자에 입력합니다
        output = netD(fake.detach()).view(-1)
        # 손실값을 구합니다
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        # 생성된 이미지와 실제 이미지에서 구한 손실값을 더합니다
        errD = errD_real + errD_fake
        # 판별자를 업데이트 합니다
        optimizerD.step()

        ############
        # 생성자 학습 #
        ############
        netG.zero_grad()
        label.fill_(real_label)
        # 판별자에 생성된 이미지를 입력합니다
        output = netD(fake).view(-1)
        # 생성자의 손실값을 구합니다
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        # 생성자를 업데이트 합니다
        optimizerG.step()

        ###############
        # 훈련 상태를 출력#
        ###############
        if i % 50 == 0:
            print(
                "[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f"
                % (
                    epoch,
                    num_epochs,
                    errD.item(),
                    errG.item(),
                    D_x,
                    D_G_z1,
                    D_G_z2,
                )
            )

        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # 고정해둔 SEED값을 사용하여 모델이 학습하는 과정을 확인합니다
        if (iters % 500 == 0) or (
            (epoch == num_epochs - 1) and (i == len(dataloader) - 1)
        ):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

학습이 완료되었다면 결과를 확인해볼까요? 생성자와 판별자의 Loss 변화를 그래프를 통해 살펴봅시다.

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses, label="G")
plt.plot(D_losses, label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

이번엔 모델이 생성한 이미지를 epoch가 진행됨에 따라 달라지는 것을 살펴볼까요?

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

만족스러운 결과인가요? DCGAN은 실습 처음에 말씀드렸듯이 이후의 GAN 모델의 기본이 되는 모델입니다. 이후 실습을 통해 GAN 모델의 발전을 살펴봅시다.

## 2. CGAN

Vanilla GAN은 무작위 노이즈(z)를 입력받아 데이터를 생성하는 모델입니다. CGAN(Conditional Generative Adversarial Networks)은 **추가적인 조건이나 정보**(Condition)를 노이즈와 함께 입력받아 데이터 생성하는 모델입니다. 이때 Contidion은 원하는 특성이나 레이블을 의미합니다. 생성자는 이 Condition에 맞고 실제와 비슷한 데이터를 생성하는 것을 목표로 하고, 판별자는 생성된 이미지가 실제 이미지인지와 더불이 Condition에 맞는 데이터인지 판별합니다.

<center><img src="img/CGAN.png" style="width: 400px"></center>

In [3]:
import pandas as pd
from PIL import Image
from torch.autograd import Variable
from torchvision.utils import make_grid
from tqdm import tqdm

### 2.1 데이터 세트 불러오기

사용하는 데이터는 Fashion MNIST 데이터입니다. Fashion MNIST는 MNIST 데이터 세트를 보완하기 위해 만들어진 데이터 세트로 패션 제품의 이미지 70,000장으로 이루어져 있습니다. 각 이미지는 28x28 픽셀의 흑백 이미지로, 10개의 클래스(티셔츠, 바지, 재킷, 드레스, 코트, 샌달, 셔츠, 스니커즈, 가방,앵클 부트)에 따라 이미지를 분류합니다.

<center><img src="img/Fashion_MNIST.png" style="width: 600px"></center>

In [4]:
class FashionMNIST(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        fashion_df = pd.read_csv("./fashion-mnist_train.csv")
        self.labels = fashion_df.label.values
        self.images = fashion_df.iloc[:, 1:].values.astype("uint8").reshape(-1, 28, 28)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = self.labels[idx]
        img = Image.fromarray(self.images[idx])

        if self.transform:
            img = self.transform(img)

        return img, label

Fashion MNIST 데이터 세트를 한번 살펴볼까요? 데이터 세트의 첫 번째 이미지를 불러오겠습니다.

In [6]:
dataset = FashionMNIST()
print(dataset[0])
dataset[0][0]

(<PIL.Image.Image image mode=L size=28x28 at 0x1DBC4F9F9B0>, 2)


위에서 불러온 이미지와 같이 Fashion MNIST 데이터 세트는 10개의 클래스로 분류되는 흑백 의류 이미지입니다.

DCGAN에서와 비슷하게 이 데이터 세트를 모델 학습에 적합한 형태로 변환하겠습니다.

In [7]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, ), std=(0.5, ))
])
dataset = FashionMNIST(transform=transform)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

### 2.2 생성자

DCGAN의 생성자와 가장 큰 차이점은 노이즈 뿐만 아니라 `label`을 입력 받아 학습에서 사용한다는 점입니다.

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()

        # label을 임베딩합니다. (10개의 클래 스를 10차원의 벡터로)
        self.label_emb = nn.Embedding(10, 10)

        # 모델 구조
        self.model = nn.Sequential(
            nn.Linear(110, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )

    def forward(self, z, labels):
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels)
        
        # 노이즈(z)와 label(c)를 연결하여 생성자가 이미지생성시 label도 사용하도록 합니다.
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out.view(x.size(0), 28, 28)

In [ ]:
generator = Generator().cuda()

### 2.3 판별자

생성자와 마찬가지로 판별자 역시 입력으로 이미지 데이터와 `label`을 받습니다.

In [13]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        # label을 임베딩합니다. (10개의 클리스를 10차원의 벡터로)
        self.label_emb = nn.Embedding(10, 10)

        # 모델 구조
        self.model = nn.Sequential(
            nn.Linear(794, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x, labels):
        x = x.view(x.size(0), 28*28)
        c = self.label_emb(labels)
        
        # 입력받은 이미지(x)와 label(c)를 연결합니다.
        x = torch.cat([x, c], 1)
        out = self.model(x)
        return out.squeeze()

In [ ]:
discriminator = Discriminator().cuda()

### 1.4 손실함수와 옵티마이저

이제 손실함수와 옵티마이저를 설정합니다. CGAN은 손실함수로 `Binary Cross Entropy loss`를 사용합니다.

In [15]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-4)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)

### 2.4 학습

이제 위에서 정의한 생성자, 판별자, 그리고 손실함수와 옵티마이저를 사용해서 CGAN을 학습해 보도록 하겠습니다. `generator_train_step`함수와 `discriminator_train_step`함수는 각각 생성자와 판별자의 학습 단계를 구현한 함수입니다.

In [16]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(
        torch.LongTensor(np.random.randint(0, 10, batch_size))
    ).cuda()
    
    # 노이즈(z)와 라벨(fake_labels)를 통해 이미지를 생성합니다.
    fake_images = generator(z, fake_labels)
    
    # 생성한 이미지를 판별자에 입력합니다.
    validity = discriminator(fake_images, fake_labels)
    
    # 손실값을 구합니다.
    g_loss = criterion(validity, Variable(torch.ones(batch_size)).cuda())
    g_loss.backward()
    g_optimizer.step()
    return g_loss.item()

In [17]:
def discriminator_train_step(
    batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels
):
    d_optimizer.zero_grad()

    # 실제 이미지를 판별자에 입력합니다.
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, Variable(torch.ones(batch_size)).cuda())

    # 생성한 이미지를 판별자에 입력합니다.
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(
        torch.LongTensor(np.random.randint(0, 10, batch_size))
    ).cuda()
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, Variable(torch.zeros(batch_size)).cuda())

    # 실제 이미지와 생성한 이미지의 손실값을 더합니다.
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss.item()

이제 실제로 학습을 진행해 보겠습니다. 학습에는 10~15분정도 소요됩니다.

In [ ]:
num_epochs = 30
n_critic = 5
display_step = 300

print("Starting Training Loop...")
for epoch in range(num_epochs):
    print("Starting epoch {}...".format(epoch))
    for i, (images, labels) in tqdm(enumerate(data_loader)):
        real_images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        generator.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(
            len(real_images),
            discriminator,
            generator,
            d_optimizer,
            criterion,
            real_images,
            labels,
        )

        g_loss = generator_train_step(
            batch_size, discriminator, generator, g_optimizer, criterion
        )

    generator.eval()
    print("g_loss: {}, d_loss: {}".format(g_loss, d_loss))

어떤가요? 꽤 만족스러운 결과를 얻을 수 있습니다.

하지만 CGAN에서 중요한 점은 실제 이미지와 유사한 이미지를 생성하는 것 외에도 조건(Condition)에 맞는 이미지를 생성하는 것입니다. 아래 코드를 실행하여 조건에 맞는 이미지를 생성하는지 확인해 봅시다.

In [ ]:
z = Variable(torch.randn(100, 100)).cuda()
labels = Variable(torch.LongTensor([i for _ in range(10) for i in range(10)])).cuda()
sample_images = generator(z, labels).unsqueeze(1).data.cpu()
grid = make_grid(sample_images, nrow=10, normalize=True).permute(1, 2, 0).numpy()
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(grid)
_ = plt.yticks([])
_ = plt.xticks(
    np.arange(15, 300, 30),
    [
        "T-Shirt",
        "Trouser",
        "Pullover",
        "Dress",
        "Coat",
        "Sandal",
        "Shirt",
        "Sneaker",
        "Bag",
        "Ankle boot",
    ],
    rotation=45,
    fontsize=20,
)

각 카테고리별로 유사한 형태를 띄는 것을 확인할 수 있습니다.

---

<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>